In [1]:
from ortools.sat.python import cp_model
import math

In [6]:



def calculate_distance(coord1, coord2, satellite_speed):

    lat1, lon1 = math.radians(coord1[0]), math.radians(coord1[1])
    lat2, lon2 = math.radians(coord2[0]), math.radians(coord2[1])
    
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    r = 6371  # Radius of earth in kilometers
    
    return int(c * r / satellite_speed)


def satellite_solver(satellite, requests):
    model = cp_model.CpModel()
    
    num_requests = len(requests)
    
    capture_durations = []
    mem_usages = []
    
    for req in requests:
        # Image capture duration
        duration = min(req["area_size_km2"] * satellite["image_duration_per_km2_sec"], 
                       satellite["max_photo_duration_s"])

        capture_durations.append(int(duration))
        
        # Total memory usage
        memory = req["area_size_km2"] * satellite["image_size_per_km2_gb"]
        
        mem_usages.append(memory)
    
    # Decision variables (if the satellite is chosen and its start time)
    is_selected = [model.NewBoolVar(f"select_{i}") for i in range(num_requests)]
    start_times = [model.NewIntVar(req["time_window_sec"][0], req["time_window_sec"][1], f"start_{i}") 
                   for i, req in enumerate(requests)]
    
    # Travel time of each pair of points
    travel_times = {}
    for i in range(num_requests):
        for j in range(num_requests):
            if i != j:
                travel_times[i, j] = calculate_distance(
                    requests[i]["coordinates"],
                    requests[j]["coordinates"],
                    satellite['speed_kms_per_s']
                )
    
    # Calculate the latest ending time from all time windows (with recalibration)
    horizon = max(req["time_window_sec"][1] for req in requests) + satellite["recalibration_time_s"]
    
    # Calculate end times for each task
    end_times = []
    for i in range(num_requests):
        end_time = model.NewIntVar(0, horizon, f"end_{i}")
        model.Add(end_time == start_times[i] + capture_durations[i]).OnlyEnforceIf(is_selected[i])
        
        # If not selected, sets a default value
        model.Add(end_time == 0).OnlyEnforceIf(is_selected[i].Not())
        end_times.append(end_time)
    
    # Sequence of selected location, checks if i is visited before j or j before i
    sequence = {}
    for i in range(num_requests):
        for j in range(i+1, num_requests):
            sequence[i, j] = model.NewBoolVar(f"sequence_{i}_{j}")
    
    # Multiple selected requests
    for i in range(num_requests):
        for j in range(i+1, num_requests):
            # i before j
            model.Add(
                start_times[j] >= start_times[i] + capture_durations[i] + satellite["recalibration_time_s"] + travel_times[i, j]
            ).OnlyEnforceIf([is_selected[i], is_selected[j], sequence[i, j]])
            
            # j before i
            model.Add(
                start_times[i] >= start_times[j] + capture_durations[j] + satellite["recalibration_time_s"] + travel_times[j, i]
            ).OnlyEnforceIf([is_selected[i], is_selected[j], sequence[i, j].Not()])
            
    # Ensure time windows
    for i, req in enumerate(requests):
        model.Add(start_times[i] + capture_durations[i] <= req["time_window_sec"][1]).OnlyEnforceIf(is_selected[i])
    
    # Memory limit of the satellite
    # Scaling the float to an int
    scale = 1000 
    memory_capacity_scaled = int(satellite["memory_capacity_gb"] * scale)
    mem_usages_scaled = [int(mem * scale) for mem in mem_usages]
    
    model.Add(sum(mem_usages_scaled[i] * is_selected[i] for i in range(num_requests)) <= memory_capacity_scaled)
    
    # The objective is to maxssimize the total priority of images
    priority_score = sum(requests[i]["priority"] * is_selected[i] for i in range(num_requests))
    model.Maximize(priority_score)
    
    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    
    results = []
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        
        selected_indices = [i for i in range(num_requests) if solver.Value(is_selected[i])]
        selected_indices.sort(key=lambda i: solver.Value(start_times[i]))
        
        total_memory = 0
        
        # Selected locations
        for idx, i in enumerate(selected_indices):
            start = solver.Value(start_times[i])
            duration = capture_durations[i]
            memory = mem_usages[i]
            total_memory += memory
            
            travel_time = 0
            if idx > 0:
                prev_idx = selected_indices[idx-1]
                travel_time = calculate_distance(requests[prev_idx]["coordinates"], requests[i]["coordinates"], satellite['speed_kms_per_s'])
            
            result = {
                "location": requests[i]["location"],
                "priority": requests[i]["priority"],
                "start_time": start,
                "duration": duration,
                "end_time": start + duration + travel_time,
                "memory_used": memory,
                "travel_time": travel_time,
                "selected": True,
                "time_window": requests[i]["time_window_sec"]
            }
            results.append(result)
        
        # Unselected locations
        for i in range(num_requests):
            if i not in selected_indices:
                results.append({
                    "location": requests[i]["location"],
                    "priority": requests[i]["priority"],
                    "selected": False
                })
    else:
        # No solutions
        for i in range(num_requests):
            results.append({
                "location": requests[i]["location"],
                "priority": requests[i]["priority"],
                "selected": False
            })
    
    return status, results

satellite = {
    "memory_capacity_gb": 5,
    "image_size_per_km2_gb": 0.15,
    "image_duration_per_km2_sec": 3.5,
    "max_photo_duration_s": 120,
    "simultaneous_tasks": False,
    "recalibration_time_s": 30,
    "speed_kms_per_s": 50
}

requests = [
    {"location": "Tokyo", "coordinates": (35.6895, 139.6917), "priority": 3, "area_size_km2": 10, "time_window_sec": (0, 1000)},
    {"location": "Paris", "coordinates": (48.8566, 2.3522), "priority": 1, "area_size_km2": 8, "time_window_sec": (1000, 1500)},
    {"location": "Montréal", "coordinates": (45.5017, -73.5673), "priority": 2, "area_size_km2": 8, "time_window_sec": (500, 1200)},
    {"location": "New-York", "coordinates": (40.730610, -73.935242), "priority": 3, "area_size_km2": 6, "time_window_sec": (500, 1200)}
]


status, results = satellite_solver(satellite, requests)

if status == cp_model.OPTIMAL:
    print("Found optimal solution!")
elif status == cp_model.FEASIBLE:
    print("Found a feasible solution.")
else:
    print("No solution found.")

selected_results = [r for r in results if r.get("selected", True)]
print(f"\nScheduled {len(selected_results)} out of {len(requests)} image captures:")

total_memory = 0
total_priority = 0
prev_end_time = 0

for idx, r in enumerate(selected_results):
    if r["selected"]:
        print(f"{r['location']} (Priority {r['priority']}): Start at {r['start_time']}s, Duration: {r['duration']}s, Time window: {r['time_window']}")
        print(f"  Memory used: {r['memory_used']:.2f} GB, Travel time from previous: {r['travel_time']}s")
        if idx == len(selected_results) - 1:
            effective_end = r['start_time'] + r['duration']
            
            print(f"  No recalibration.")
        else:
            effective_end = r['start_time'] + r['duration'] + satellite['recalibration_time_s']
            
            print(f"  Recalibration time: {satellite['recalibration_time_s']}s")
        # end_time = r['start_time'] + r['duration'] + r['travel_time'] + satellite['recalibration_time_s'] * (idx != len(selected_results) - 1)
        # print(f"  End task at: {end_time}s")
        print(f"  End task at: {effective_end}s")
        
        
        # if prev_end_time > 0:
            # print(f"  Time since previous task: {r['start_time'] - prev_end_time}s")
            
        if idx > 0:
            print(f"  Time since previous task: {r['start_time'] - prev_end_time}s")
        
        # prev_end_time = end_time
        prev_end_time = effective_end
        
        print()
        
        total_memory += r['memory_used']
        total_priority += r['priority']
        
print(f"Total memory used: {total_memory:.2f} GB out of {satellite['memory_capacity_gb']} GB")
print(f"Total priority score: {total_priority}")

unscheduled = [r for r in results if not r.get("selected", True)]
if unscheduled:
    print("\nUnscheduled locations:")
    for r in unscheduled:
        print(f"{r['location']} (Priority {r['priority']})")


Found optimal solution!

Scheduled 4 out of 4 image captures:
New-York (Priority 3): Start at 500s, Duration: 21s, Time window: (500, 1200)
  Memory used: 0.90 GB, Travel time from previous: 0s
  Recalibration time: 30s
  End task at: 551s

Montréal (Priority 2): Start at 561s, Duration: 28s, Time window: (500, 1200)
  Memory used: 1.20 GB, Travel time from previous: 10s
  Recalibration time: 30s
  End task at: 619s
  Time since previous task: 10s

Tokyo (Priority 3): Start at 826s, Duration: 35s, Time window: (0, 1000)
  Memory used: 1.50 GB, Travel time from previous: 207s
  Recalibration time: 30s
  End task at: 891s
  Time since previous task: 207s

Paris (Priority 1): Start at 1085s, Duration: 28s, Time window: (1000, 1500)
  Memory used: 1.20 GB, Travel time from previous: 194s
  No recalibration.
  End task at: 1113s
  Time since previous task: 194s

Total memory used: 4.80 GB out of 5 GB
Total priority score: 9


In [9]:
import math
import numpy as np
import ortools
from ortools.sat.python import cp_model

class Satellite:
    def __init__(self, memory_capacity_gb, image_size_per_km2_gb, image_duration_per_km2_sec, max_photo_duration_s, recalibration_time_s,  speed_kms_per_s):
        self.memory_capacity_gb = memory_capacity_gb
        self.image_size_per_km2_gb = image_size_per_km2_gb
        self.image_duration_per_km2_sec = image_duration_per_km2_sec
        self.max_photo_duration_s = max_photo_duration_s
        self.recalibration_time_s = recalibration_time_s
        self.speed_kms_per_s = speed_kms_per_s
    
    def calculate_distance(self, coord1, coord2):
        lat1, lon1 = math.radians(coord1[0]), math.radians(coord1[1])
        lat2, lon2 = math.radians(coord2[0]), math.radians(coord2[1])
        
        # Haversine formula
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
        c = 2 * math.asin(math.sqrt(a))
        r = 6371
        return int(c * r / self.speed_kms_per_s)
        
    def calculate_travel_time(self, coord1, coord2):
        distance = self.calculate_distance(coord1, coord2)
        return distance / self.speed_kms_per_s

    def calculate_memory_usage(self, area_size_km2):
        return area_size_km2 * self.image_size_per_km2_gb
    
    def calculate_capture_duration(self, area_size_km2):
        duration = min(area_size_km2 * self.image_duration_per_km2_sec, self.max_photo_duration_s)
        return int(duration)


class Request:
    def __init__ (self, location, coordinates, priority, area_size_km2, time_window_sec):
        self.location = location
        self.coordinates = coordinates
        self.priority = priority
        self.area_size_km2 = area_size_km2
        self.time_window_sec = time_window_sec


class SatelliteScheduler:
    def __init__(self, satellite: Satellite, requests: list[Request]):
        self.satellite = satellite
        self.requests = requests
        self.model = cp_model.CpModel()
        self.solver = cp_model.CpSolver()
        self.capture_durations = []
        self.mem_usages = []
        self.execution_log = []

    def solve(self):

        n = len(self.requests)

        for req in self.requests:
            # Image capture duration
            duration = self.satellite.calculate_capture_duration(req.area_size_km2)
            self.capture_durations.append(int(duration))

            # Total memory usage
            memory = self.satellite.calculate_memory_usage(req.area_size_km2)
            self.mem_usages.append(memory)

        # Decision variables (if the satellite is chosen and its start time)
        is_selected = [self.model.NewBoolVar(f"select_{i}") for i in range(n)]
        start_times = [self.model.NewIntVar(req.time_window_sec[0], req.time_window_sec[1], f"start_{i}")
                          for i, req in enumerate(self.requests)]
        
        # Travel time of each pair of points
        travel_times = {}
        for i in range(n):
            for j in range(n):
                if i != j:
                    travel_times[i, j] = self.satellite.calculate_travel_time(
                        self.requests[i].coordinates,
                        self.requests[j].coordinates
                    )

        # Calculate the latest ending time from all time windows (with recalibration)
        horizon = max(req.time_window_sec[1] for req in self.requests) + self.satellite.recalibration_time_s

        # Calculate the end times for each task
        end_times = []
        horizon = max(req.time_window_sec[1] for req in self.requests) + self.satellite.recalibration_time_s
        for i in range(n):
            end_time = self.model.NewIntVar(0, horizon, f"end_{i}")
            self.model.Add(end_time == start_times[i] + self.capture_durations[i]).OnlyEnforceIf(is_selected[i])

            # If not selected, sets a default value
            self.model.Add(end_time == 0).OnlyEnforceIf(is_selected[i].Not())
            end_times.append(end_time)

        # Sequence of selected location, checks if i is visited before j or j before i
        sequence = {}
        for i in range(n):
            for j in range(i+1, n):
                sequence[i, j] = self.model.NewBoolVar(f"sequence_{i}_{j}")

        # Multiple selected requests
        for i in range(n):
            for j in range(i+1, n):
                # i before j
                self.model.Add(start_times[j] >= start_times[i] + self.capture_durations[i] + self.satellite.recalibration_time_s + travel_times[i, j]).OnlyEnforceIf([is_selected[i], is_selected[j], sequence[i, j]])

                # j before i
                self.model.Add(start_times[i] >= start_times[j] + self.capture_durations[j] + self.satellite.recalibration_time_s + travel_times[j, i]).OnlyEnforceIf([is_selected[i], is_selected[j], sequence[i, j].Not()])

    
        # Ensure time windows
        for i, req in enumerate(self.requests):
            self.model.Add(start_times[i] + self.capture_durations[i] <= req.time_window_sec[1]).OnlyEnforceIf(is_selected[i])

        # Memory constraint
        # Memory limit of the satellite
        # Scaling the float to an int
        scale = 1000 
        memory_capacity = int(self.satellite.memory_capacity_gb * scale)
        memory_usage = [int(mem * scale) for mem in self.mem_usages]
        self.model.Add(sum(memory_usage[i] * is_selected[i] for i in range(n)) <= memory_capacity)

        # Objective function: maximize the number of selected requests
        priority_score = sum(req.priority * is_selected[i] for i, req in enumerate(self.requests))
        self.model.Maximize(priority_score)

        # Solve the model
        status = self.solver.Solve(self.model)

        results = []
        if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
            
            selected_indices = [i for i in range(n) if self.solver.Value(is_selected[i]) == 1]
            selected_indices.sort(key=lambda i: self.solver.Value(start_times[i]))

            total_memory = 0

            # Selected locations
            for idx, i in enumerate(selected_indices):
                start = self.solver.Value(start_times[i])
                duration = self.capture_durations[i]
                memory = self.mem_usages[i]
                total_memory += memory

                travel_time = 0
                if idx > 0:
                    prev_idx = selected_indices[idx - 1]
                    travel_time = self.satellite.calculate_distance(
                        self.requests[prev_idx].coordinates,
                        self.requests[i].coordinates,
                    )

                    result = {
                        "location": self.requests[i].location,
                        "priority": self.requests[i].priority,
                        "start_time": start,
                        "duration": duration,
                        "end_time": start + duration + travel_time,
                        "memory_used": memory,
                        "travel_time": travel_time,
                        "selected": True,
                        "time_window": self.requests[i].time_window_sec,
                    }
                    results.append(result)

            # Unselected locations
            for i in range(n):
                if i not in selected_indices:
                    result = {
                        "location": self.requests[i].location,
                        "priority": self.requests[i].priority,
                        "selected": False,
                    }
                    results.append(result)
        else:
            # No solution found
            for i in range(n):
                result = {
                    "location": self.requests[i].location,
                    "priority": self.requests[i].priority,
                    "selected": False,
                }
                results.append(result)
    
        return status, results
    
    def print_solution(self, status, results):
        if status == cp_model.OPTIMAL:
            print("Found optimal solution!")
        elif status == cp_model.FEASIBLE:
            print("Found a feasible solution.")
        else:
            print("No solution found.")

        selected_results = [r for r in results if r.get("selected", True)]
        print(f"\nScheduled {len(selected_results)} out of {len(self.requests)} image captures:")

        total_memory = 0
        total_priority = 0
        prev_end_time = 0

        for idx, r in enumerate(selected_results):
            if r["selected"]:
                print(f"{r['location']} (Priority {r['priority']}): Start at {r['start_time']}s, Duration: {r['duration']}s, Time window: {r['time_window']}")
                print(f"  Memory used: {r['memory_used']:.2f} GB, Travel time from previous: {r['travel_time']}s")
                if idx == len(selected_results) - 1:
                    effective_end = r['start_time'] + r['duration']
                    
                    print(f"  No recalibration.")
                else:
                    effective_end = r['start_time'] + r['duration'] + self.satellite.recalibration_time_s
                    
                    print(f"  Recalibration time: {self.satellite.recalibration_time_s}s")
                print(f"  End task at: {effective_end}s")
                
                if idx > 0:
                    print(f"  Time since previous task: {r['start_time'] - prev_end_time}s")
                
                prev_end_time = effective_end
                
                print()
                
                total_memory += r['memory_used']
                total_priority += r['priority']
                
        print(f"Total memory used: {total_memory:.2f} GB out of {self.satellite.memory_capacity_gb} GB")
        print(f"Total priority score: {total_priority}")

        unscheduled = [r for r in results if not r.get("selected", True)]
        if unscheduled:
            print("\nUnscheduled locations:")
            for r in unscheduled:
                print(f"{r['location']} (Priority {r['priority']})")

                
if __name__ == "__main__":
    # Example usage
    satellite = Satellite(
        memory_capacity_gb=100,
        image_size_per_km2_gb=0.1,
        image_duration_per_km2_sec=5,
        max_photo_duration_s=300,
        recalibration_time_s=60,
        speed_kms_per_s=10
    )

    requests = [
        Request("New-York", (40.730610, -73.935242), 3, 6, (500, 1200)),
        Request("Los-Angeles", (34.052235, -118.243683), 5, 8, (600, 1300)),
        Request("Chicago", (41.878113, -87.629799), 2, 4, (700, 1400)),
        Request("San-Francisco", (37.774929, -122.419416), 4, 5, (800, 1500)),
        Request("Miami", (25.761680, -80.191790), 3, 7, (400, 1100)),
        Request("Seattle", (47.608013, -122.335167), 4, 6, (900, 1600)),
        Request("Houston", (29.760427, -95.369803), 2, 5, (600, 1300)),
        Request("Boston", (42.360081, -71.058880), 3, 4, (450, 1150))
    ]

    scheduler = SatelliteScheduler(satellite, requests)
    status, results = scheduler.solve()
    scheduler.print_solution(status, results)
    

        

TypeError: Linear constraints only accept integer values and coefficients: start_1(600..1300) and FloatAffine(expr=IntAffine(expr=start_0(500..1200), coeff=1, offset=90), coeff=1, offset=39.4)